In this notebook we show how to prepare all the input and parameter files necessary to run Diva4D.<br>
The code rely on the classes defined for 2D (module `pydiva2d`) and 4D (module `pydiva4d`). 

In [ ]:
import os
import logging
import shutil
import subprocess
import pydiva4d
import datetime
from importlib import reload

In [2]:
# reload(pydiva4d)

Reload the module (only during code development), (un)comment if necessary.

# User inputs and configuration

Indicate the path to the Diva installation you want to work with (ending with `diva-x.y.z`).

In [3]:
divabasedir = "/home/ctroupin/Software/DIVA/diva-4.7.1"

## Create the sub-directories and files

In [4]:
divadirs = pydiva4d.DivaDirectories(divabasedir)

INFO:pydiva2d:Diva main directory: /home/ctroupin/Software/DIVA/diva-4.7.1
INFO:pydiva2d:Creating Diva directory paths
INFO:pydiva2d:Binary directory:   /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/bin
INFO:pydiva2d:Source directory:   /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/src/Fortan
INFO:pydiva2d:Main 2D directory:  /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped
INFO:pydiva2d:Main 4D directory:  /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology
INFO:pydiva2d:4D input directory: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/input


In [5]:
diva2Dfiles = pydiva4d.Diva2Dfiles(divadirs.diva2d)

INFO:pydiva2d:Creating Diva 2D file names and paths
INFO:pydiva2d:Contour file:   /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/coast.cont
INFO:pydiva2d:Parameter file: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/param.par
INFO:pydiva2d:Data file:      /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/data.dat
INFO:pydiva2d:Valatxy file:   /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/input/valatxy.coord
INFO:pydiva2d:Result file:    /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/output/ghertonetcdf/results.nc
INFO:pydiva2d:Mesh file:      /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/meshgenwork/fort.22
INFO:pydiva2d:Mesh topo file: /home/ctroupin/Software/DIVA/diva-4.7.1/DIVA3D/divastripped/meshgenwork/fort.23


In [6]:
diva4Dfiles = pydiva4d.Diva4Dfiles(divadirs.diva4d)

INFO:pydiva4d:Creating Diva 4D file names and paths
INFO:pydiva4d:datasource file:   /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/datasource
INFO:pydiva4d:constandrefe file: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/constandrefe
INFO:pydiva4d:driver file:       /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/driver
INFO:pydiva4d:monthlist file:    /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/monthlist
INFO:pydiva4d:qflist file:       /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/qflist
INFO:pydiva4d:varlist file:      /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/varlist
INFO:pydiva4d:yearlist file:     /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/yearlist
INFO:pydiva4d:contourdepth file: /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/contourdepth
INFO:pydiva4d:ncdfinfo file:     /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/ncdfinfo
INFO:pydiva4d:param.par file:    /home/ctroupin/Sof

## List of user parameters

Set the different parameters necessary for a 4D analysis.<br>
For clarity, they are separated by input files.

### List of ODV spreadsheet files

In [7]:
datasource = pydiva4d.Datasource('./blacksea_data_CTD.txt')

INFO:pydiva4d:Creating Diva 4D Datasource object


### Parameters for the advection constraint and the reference field

In [8]:
advection_flag = 0               # indicates if advection is activated
ref_flag = 0                     # indicates if there is a reference field
var_year_code = '00000000'       # variable year code
var_month_code = '0000'          # variable month code

In [9]:
constandrefe = pydiva4d.Constandrefe(advection_flag, ref_flag, var_year_code, var_month_code)

INFO:pydiva4d:Creating Diva 4D Constandrefe object


### Parameters for the 4D analysis

In [10]:
extraction_flag = 1              # Data extraction: 1 do it, 0 do nothing, -1 press coord, -10 pressure+Saunders
coast_flag = 1                   # Boundary lines and coastlines generation: 0 nothing, 1: contours, 2: UV, 3: 1+2
clean_flag = 0                   # Cleaning data on mesh: 1, 2: RL, 3: both, 4: 1 + outliers elimination, 5: =4+2
min_datanum = 0                  # Minimal number of data in a layer. If less, uses data from any month.
param_flag = -30                 # Parameters estimation and vertical filtering
min_l, max_l = 1.4, 10           # Minimal/maximal correlation length during optimization
min_snr, max_snr = 0.1, 5.0      # Minimal/maximal signal-to-noise ratio during optimization
analysisref_flag = 1             # Creation of reference field: 2 do reference, 1 do analysis and 0 do nothing
lower_level, upper_level = 1, 3  # Depth lower/upper level numbers
netcdf4d_flag = 0                # 4D netcdf files: 0 (no generation) or 1
gnuplot_flag = 0                 # Gnuplots generation: 0 (no generation) or 1
detrend_groupnum = 0             # Data detrending: number of groups, 0 if no detrending.

In [11]:
driver = pydiva4d.Driver(extraction_flag, coast_flag, clean_flag, min_datanum, param_flag, min_l,\
                               max_l, min_snr, max_snr, analysisref_flag, lower_level,\
                               upper_level, netcdf4d_flag, gnuplot_flag, detrend_groupnum)

INFO:pydiva4d:Creating Diva 4D Driver object


### Parameters for the netCDF file

In [12]:
# Title string for 3D NetCDF file
ncdf_title = '4D Test on the Black Sea'
# Reference time for data (ie: days since since 1900-01-01), if not climatological data
ncdf_reftime = 'months since since xxxx-01-01'
# Time value (that reprsents the data set), if not climatological data
ncdf_timeval = 1200
# Cell_methode string
ncdf_cellmethod = 'time: mean (this month data from all years)'
# Institution name: where the dataset was produced.
ncdf_institution = 'University of Liege, GeoHydrodynamics and Environment Research'
# Production group and e-mail
ncdf_groupemail = 'Diva group. E-mails : a.barth@ulg.ac.be ; swatelet@ulg.ac.be'
# Source (observation, radiosonde, database, model-generated data,...)
ncdf_source = 'data_from various sources for diva software testing work'
ncdf_comment = 'No comment'
ncdf_authoremail = 'swatelet@ulg.ac.be'
ncdf_acknowlegment = 'No acknowledgement'

In [13]:
netcdfinfo = pydiva4d.Ncdfinfo(ncdf_title, ncdf_reftime, ncdf_timeval, ncdf_cellmethod,\
                                     ncdf_institution, ncdf_groupemail, ncdf_source,\
                                     ncdf_comment, ncdf_authoremail, ncdf_acknowlegment)

INFO:pydiva4d:Creating Diva 4D Ncdfinfo object


### Parameters for the output grid

In [14]:
xmin, ymin = 27.0, 40.0        # longitute and latitude of the lower-left corner of the domain
nx, ny = 151, 76               # number of cells in the x and y directions
dx, dy = 0.1, 0.1              # spatial steps in the x and y directions

### Parameters for the analysis

In [15]:
CorrelationLength = 1.5
SignalToNoiseRatio = 0.5
VarianceBackgroundField = 1.0
ExclusionValue = -99.
iCoordChange = 2
iSpec = 11
iReg = 2

In [16]:
parameters2D = pydiva4d.Diva2DParameters(CorrelationLength, iCoordChange, iSpec, iReg, xmin, ymin, dx, dy, nx, ny,
                                ExclusionValue, SignalToNoiseRatio, VarianceBackgroundField)

INFO:pydiva2d:Creating Diva 2D parameter object


## Others: monthlist, QC flags, variables, years, contour levels

In [17]:
months = pydiva4d.Monthlist(['0101', '0202', '0303'])       # list of strings representing the periods (months)
qflags = pydiva4d.Qflist([0, 1])                            # list of integers representing the accepted flags
variables = pydiva4d.Varlist(['Temperature', 'Salinity'])   # list of strings representing the variables to extract
years = pydiva4d.Yearlist(['19002009',])                    # list of strings representing the perior (years)
contours = pydiva4d.Contourdepth([30., 20., 10., 0.])       # list of floats representing the vertical levels

INFO:pydiva4d:Creating Diva 4D Monthlist object
INFO:pydiva4d:Creating Diva 4D Qflist object
INFO:pydiva4d:Creating Diva 4D Varlist object
INFO:pydiva4d:Creating Diva 4D Yearlist object
INFO:pydiva4d:Creating Diva 4D Contourdepth object


# Create the new input files

Now we have set all the parameters, we can write the information into the corresponding Diva files.<br>
All the objects have a function *write_to* that is used to write to a file.

## Write the different input files

In [18]:
datasource.write_to(diva4Dfiles.datasource)

INFO:pydiva4d:Written into file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/datasource


In [19]:
constandrefe.write_to(diva4Dfiles.constandrefe)

INFO:pydiva4d:Written into file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/constandrefe


In [20]:
driver.write_to(diva4Dfiles.driver)

INFO:pydiva4d:Written into file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/driver


In [21]:
months.write_to(diva4Dfiles.monthlist)

INFO:pydiva4d:Written in file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/monthlist


In [22]:
qflags.write_to(diva4Dfiles.qflist)

INFO:pydiva4d:Written in file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/qflist


In [23]:
variables.write_to(diva4Dfiles.varlist)

INFO:pydiva4d:Written in file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/varlist


In [24]:
years.write_to(diva4Dfiles.yearlist)

INFO:pydiva4d:Written into file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/yearlist


In [25]:
netcdfinfo.write_to(diva4Dfiles.ncdfinfo)

INFO:pydiva4d:Written into file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/ncdfinfo


In [26]:
parameters2D.write_to(diva4Dfiles.param)

INFO:pydiva2d:Written parameters into file /home/ctroupin/Software/DIVA/diva-4.7.1/JRA4/Climatology/input/param.par


# Run divadoall (4D analysis)

We call the main script `divadoall` to perform the 4D analysis.<br>
The time for the analysis depends on the number of variables, levels, periods, ...

In [27]:
pydiva4d.logger.info("Starting to run 'divadoall'")
divadoall_script = './divadoall'
env = os.environ.copy()
env["PATH"] = ".:" + env["PATH"]

p = subprocess.Popen(divadoall_script, stdout=subprocess.PIPE, cwd=divadirs.diva4d, env=env)
out = p.stdout.read()

INFO:pydiva4d:Starting to run 'divadoall


Write the Diva execution log into the same log file

In [29]:
pydiva4d.logger.info("Diva execution log:")
with open(pydiva4d.logfile, 'w') as f:
    f.write(str(out).replace('\\n', '\n'))

INFO:pydiva4d:Diva execution log:


In [31]:
pydiva4d.logger.info("Finished the 'divadoall' run")

INFO:pydiva4d:Finished the 'divadoall' run
